<a href="https://colab.research.google.com/github/ebinjomonkottakal/Tensorflow_advanced/blob/main/Basic_Mirror_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

# Mirrored Strategy: Basic

In [1]:
# imports 
import tensorflow as tf
import tensorflow_datasets as tfds
import os
tfds.disable_progress_bar()

Load the MNIST dataset and split it into training and test chunks.

In [2]:
datasets,info = tfds.load('mnist',with_info = True,as_supervised = True)
mnist_train,mnist_test = datasets['train'], datasets['test']

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [3]:
# Define the strategy to use and print the number of devices found
strategy = tf.distribute.MirroredStrategy()
print("Numeber of GPU's: {}".format(strategy.num_replicas_in_sync))

Numeber of GPU's: 1


In [4]:
# Get the number of examples in the train and test sets
num_train_examples = info.splits['train'].num_examples
num_test_examples = info.splits['test'].num_examples

BUFFER_SIZE = 10000
BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [5]:
# Normalise image
def scale(img,label):
  image = tf.cast(img,tf.float32)
  image/=225
  return img,label

In [6]:
# Set up train and eval dataset
train_dataset = mnist_train.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [7]:
with strategy.scope():
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32,3,activation = 'relu',input_shape = (28,28,1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64,activation = 'relu'),
      tf.keras.layers.Dense(10)
  ])

In [9]:
model.compile(loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),optimizer = tf.keras.optimizers.Adam(),metrics = ['accuracy'])

In [10]:
model.fit(train_dataset,epochs = 12)

Epoch 1/12
938/938 [==============================] - 19s 8ms/step - loss: 0.5223 - accuracy: 0.9156
Epoch 2/12
938/938 [==============================] - 4s 5ms/step - loss: 0.0940 - accuracy: 0.9732
Epoch 3/12
938/938 [==============================] - 4s 5ms/step - loss: 0.0598 - accuracy: 0.9816
Epoch 4/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0439 - accuracy: 0.9864
Epoch 5/12
938/938 [==============================] - 4s 4ms/step - loss: 0.0351 - accuracy: 0.9889
Epoch 6/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0294 - accuracy: 0.9905
Epoch 7/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0282 - accuracy: 0.9911
Epoch 8/12
938/938 [==============================] - 4s 5ms/step - loss: 0.0244 - accuracy: 0.9921
Epoch 9/12
938/938 [==============================] - 5s 5ms/step - loss: 0.0239 - accuracy: 0.9924
Epoch 10/12
938/938 [==============================] - 4s 5ms/step - loss: 0.0178 - accuracy: 0.994